<h1><font size=12>
Weather Derivatites </h1>
<h1> Rainfall Simulator -- LSTM  <br></h1>

Developed by [Jesus Solano](mailto:ja.solano588@uniandes.edu.co) <br>  
16 September 2018 


In [13]:
# Import needed libraries. 
import numpy as np
import pandas as pd
import random as rand
import matplotlib.pyplot as plt
from scipy.stats import bernoulli
from scipy.stats import gamma
import pickle
import time
import datetime
from keras.models import load_model
import os


# Generate artificial Data

In [10]:
### ENSO probabilistic forecast.

# Open saved data.
ensoForecast = pickle.load(open('../datasets/ensoForecastProb/ensoForecastProbabilities.pickle','rb'))

# Print an example .. ( Format needed)

ensoForecast['2005-01']

,Season,La Niña,Neutral,El Niño
0,JFM 2005,0%,15%,85%
1,FMA 2005,0%,17%,83%
2,MAM 2005,1%,24%,75%
3,AMJ 2005,2%,43%,55%
4,MJJ 2005,4%,51%,45%
5,JJA 2005,6%,54%,40%
6,JAS 2005,8%,52%,40%
7,ASO 2005,9%,51%,40%
8,SON 2005,10%,50%,40%
9,OND 2005,10%,50%,40%


In [11]:
### Create total dataframe.

def createTotalDataFrame(daysNumber, startDate , initialState , initialPrep , ensoForecast ):
    # Set variables names.
    totalDataframeColumns = ['state','Prep','Month','probNina','probNino', 'nextState']

    # Create dataframe.

    allDataDataframe = pd.DataFrame(columns=totalDataframeColumns) 

    # Number of simulation days(i.e 30, 60)
    daysNumber = daysNumber

    # Simulation start date ('1995-04-22')
    startDate = startDate

    # State of rainfall last day before start date --> Remember 0 means dry and 1 means wet.
    initialState = initialState
    initialPrep = initialPrep   # Only fill when initialState == 1  


    dates = pd.date_range(startDate, periods = daysNumber + 2 , freq='D')

    for date in dates: 

        # Fill precipitation amount.
        allDataDataframe.loc[date.strftime('%Y-%m-%d'),'Prep'] = np.nan

        # Fill month of date 
        allDataDataframe.loc[date.strftime('%Y-%m-%d'),'Month'] = date.month

        # Fill El Nino ENSO forecast probability. 
        allDataDataframe.loc[date.strftime('%Y-%m-%d'),'probNino'] = float(ensoForecast[date.strftime('%Y-%m')].loc[0,'El Niño'].strip('%').strip('~'))/100

        # Fill La Nina ENSO forecast probability. 
        allDataDataframe.loc[date.strftime('%Y-%m-%d'),'probNina'] = float(ensoForecast[date.strftime('%Y-%m')].loc[0,'La Niña'].strip('%').strip('~'))/100

        # Fill State. 
        allDataDataframe.loc[date.strftime('%Y-%m-%d'),'state'] = np.nan


    simulationDataFrame = allDataDataframe[:-1]

    # Fill initial conditions. 
    simulationDataFrame['state'][0] = initialState
    if initialState == 1:
        simulationDataFrame['Prep'][0] = initialPrep
    else:
        simulationDataFrame['Prep'][0] = 0.0
    
    return simulationDataFrame

simulationDataFrame = createTotalDataFrame(daysNumber= 30, startDate = '2017-08-18', initialState = 1 , initialPrep =  0.4, ensoForecast = ensoForecast)
simulationDataFrame.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,state,Prep,Month,probNina,probNino,nextState
2017-08-18,1,0.4,8,0.09,0.09,NaN
2017-08-19,NaN,NaN,8,0.09,0.09,NaN
2017-08-20,NaN,NaN,8,0.09,0.09,NaN
2017-08-21,NaN,NaN,8,0.09,0.09,NaN
2017-08-22,NaN,NaN,8,0.09,0.09,NaN


In [12]:
### Load transitions and amount parameters.

# Transitions probabilites.
transitionsParametersDry = pd.read_csv('../results/visibleMarkov/transitionsParametersDry.csv', sep = ' ', header=None, names = ['variable', 'value'])
transitionsParametersDry.index += 1 
transitionsParametersDry

transitionsParametersWet = pd.read_csv('../results/visibleMarkov/transitionsParametersWet.csv', sep = ' ', header=None, names = ['variable', 'value'])
transitionsParametersWet.index += 1 
transitionsParametersWet

amountParametersGamma = pd.read_csv('../results/visibleMarkov/amountGamma.csv', sep = ' ', header=None, names = ['variable', 'loge(mu)', 'loge(shape)'])
amountParametersGamma.index += 1 
print(amountParametersGamma)

print('\n * Intercept means firts month (January) ')

# Load neural network.
lstmModelDry = load_model('../results/visibleMarkov/rainfall_lstmDry_LSTM.h5')
# Load neural network.
lstmModelWet = load_model('../results/visibleMarkov/rainfall_lstmWet_LSTM.h5')

       variable  loge(mu)  loge(shape)
1   (Intercept)  1.473576    -0.469428
2        Month2  0.139699     0.000000
3        Month3  0.374357     0.000000
4        Month4  0.534853     0.000000
5        Month5  0.399923     0.000000
6        Month6 -0.055791     0.000000
7        Month7 -0.288876     0.000000
8        Month8 -0.277614     0.000000
9        Month9 -0.031364     0.000000
10      Month10  0.618613     0.000000
11      Month11  0.565649     0.000000
12      Month12  0.345987     0.000000
13     probNino -0.630264     0.000000
14     probNina -0.012164     0.000000

 * Intercept means firts month (January) 


InternalError: GPU sync failed

## Simulation Function Core

In [5]:
### Build the simulation core.

# Updates the state of the day based on yesterday state. 
def updateState(yesterdayIndex, simulationDataFrame, transitionsParametersDry, transitionsParametersWet):
    
    # Additional data of day.
    yesterdayState = simulationDataFrame['state'][yesterdayIndex]
    yesterdayPrep = simulationDataFrame['Prep'][yesterdayIndex]
    yesterdayProbNino = simulationDataFrame['probNino'][yesterdayIndex]
    yesterdayProbNina = simulationDataFrame['probNina'][yesterdayIndex]
    yesterdayMonth = simulationDataFrame['Month'][yesterdayIndex]

    # Calculate transition probability.
    if yesterdayState == 0:
               

        
        xPredict = np.array([(yesterdayMonth-1)/11,yesterdayProbNino,yesterdayProbNina])
        
        xPredict = np.reshape(xPredict, ( 1, 1 , xPredict.shape[0]))
        
        # Includes month factor + probNino value + probNino value.
        successProbability = lstmModelDry.predict(xPredict)[0][0]
        
    elif yesterdayState == 1:
              
        xPredict = np.array([yesterdayPrep ,(yesterdayMonth-1)/11,yesterdayProbNino,yesterdayProbNina])
        
        xPredict = np.reshape(xPredict, ( 1, 1 , xPredict.shape[0]))
        
        # Includes month factor + probNino value + probNino value.
        successProbability = lstmModelWet.predict(xPredict)[0][0] 
    else:
        print('State of date: ', simulationDataFrame.index[yesterdayIndex],' not found.')
    
    #print(successProbability)
    
    todayState = bernoulli.rvs(successProbability)
    
    return todayState 

In [6]:
# Simulates one run of simulation. 
def oneRun(simulationDataFrame, transitionsParametersDry, transitionsParametersWet, amountParametersGamma):
    
    # Define the total rainfall amount over the simulation.
    rainfall = 0
    
    # Loop over days in simulation to calculate rainfall ammount.
    for day in range(1,len(simulationDataFrame)):
        
        # Get today date. 
        dateOfDay = datetime.datetime.strptime(simulationDataFrame.index[day],'%Y-%m-%d')
       
    
        # Update today state based on the yesterday state. 
        todayState = updateState(day-1, simulationDataFrame, transitionsParametersDry, transitionsParametersWet)
        
        # Write new day information.
        simulationDataFrame['state'][day] = todayState
        simulationDataFrame['nextState'][day-1] = todayState
        
        # Computes total accumulated rainfall.
        if todayState == 1:
            
            # Additional data of day.
            todayProbNino = simulationDataFrame['probNino'][day]
            todayProbNina = simulationDataFrame['probNina'][day]
            todayMonth = simulationDataFrame['Month'][day]
    
            
            # Calculates gamma log(mu).
            gammaLogMU = amountParametersGamma['loge(mu)'][todayMonth]+ todayProbNino*amountParametersGamma['loge(mu)'][13]+todayProbNino*amountParametersGamma['loge(mu)'][13] 
            
            # Calculates gamma scale
            gammaLogShape = amountParametersGamma['loge(shape)'][1]
            
            # Update mu
            gammaMu = np.exp(gammaLogMU)
            
            # Update shape
            gammaShape = np.exp(gammaLogShape)
            
            # Calculate gamma scale.
            gammaScale = gammaMu / gammaShape
            
            # Generate random rainfall.
            todayRainfall = gamma.rvs(a = gammaShape, scale = gammaScale)
            
            # Write new day information.
            simulationDataFrame['Prep'][day] = todayRainfall
            
            # Updates rainfall amount. 
            rainfall += todayRainfall
            
        else:
            # Write new day information.
            simulationDataFrame['Prep'][day] = 0
            
        
        yesterdayState = todayState
            
        
    return rainfall

In [7]:
updateState(0, simulationDataFrame, transitionsParametersDry, transitionsParametersWet)

InternalError: Blas GEMM launch failed : a.shape=(1, 100), b.shape=(100, 100), m=1, n=100, k=100
	 [[Node: lstm_1_1/while/MatMul_7 = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](lstm_1_1/while/Switch_3:1, lstm_1_1/while/MatMul_7/Enter)]]
	 [[Node: dense_1_1/Sigmoid/_341 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_252_dense_1_1/Sigmoid", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

In [9]:
# Run only one iteration(Print structure of results)

# Simulations iterations.
iterations = 10000

oneRun(simulationDataFrame, transitionsParametersDry, transitionsParametersWet, amountParametersGamma)


20.156337891217795

## Complete Simulation

In [10]:
# Run total iterations.
def totalRun(simulationDataFrame, transitionsParametersDry, transitionsParametersWet, amountParametersGamma,iterations):
    
    # Initialize time 
    startTime = time.time()
    
    # Array to store all precipitations.
    rainfallPerIteration = [None]*iterations
    
    
    # Loop over each iteration(simulation)
    
    for i in range(iterations):
        
        simulationDataFrameC = simulationDataFrame.copy()
        
        iterationRainfall = oneRun(simulationDataFrameC, transitionsParametersDry, transitionsParametersWet, amountParametersGamma)
        
        rainfallPerIteration[i] = iterationRainfall
    
    # Calculate time
    currentTime = time.time() - startTime 
    
    # Logging time.
    print('The elapsed time over simulation is: ', currentTime, ' seconds.')
    
    return rainfallPerIteration

In [11]:
#### Define parameters simulation.

# Simulations iterations.
iterations = 10

# Create dataframe to simulate. 
simulationDataFrame = createTotalDataFrame(daysNumber= 30, startDate = '2017-08-18', initialState = 1 , initialPrep =  0.4, ensoForecast = ensoForecast)
simulationDataFrame.head()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,state,Prep,Month,probNina,probNino,nextState
2017-08-18,1,0.4,8,0.09,0.09,NaN
2017-08-19,NaN,NaN,8,0.09,0.09,NaN
2017-08-20,NaN,NaN,8,0.09,0.09,NaN
2017-08-21,NaN,NaN,8,0.09,0.09,NaN
2017-08-22,NaN,NaN,8,0.09,0.09,NaN


## Final Results

In [12]:
# Final Analysis.

finalSimulation = totalRun(simulationDataFrame, transitionsParametersDry, transitionsParametersWet, amountParametersGamma,iterations)


ResourceExhaustedError: OOM when allocating tensor with shape[100,400] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: training_207/Adam/Variable_6/Assign = Assign[T=DT_FLOAT, _grappler_relax_allocator_constraints=true, use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](training_207/Adam/Variable_6, training_207/Adam/zeros_1)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'training_207/Adam/Variable_6/Assign', defined at:
  File "/usr/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/trait

In [ ]:
fig = plt.figure(figsize=(20, 10))

plt.hist(finalSimulation,facecolor='steelblue',bins=100, density=True,
         histtype='stepfilled', edgecolor = 'black' , hatch = '+')

plt.title('Rainfall Simulation')
plt.xlabel('Rainfall Amount [mm]')
plt.ylabel('Probability ')
plt.grid()
plt.show()

# Financial Analysis